# Lesson 14 Studio Solution:  SQL Part 5

Studio notebook: https://github.com/speudusa/SQL-Part-5-Studio/blob/master/Studio.ipynb

## Part B: WARM-UP QUERIES

In [6]:
USE JunkDB_STL_May2023
GO

-- WARM UP --

-- Query 1 -- Return the mystery book titles and their ISBNs.

/* Our book titles and ISBNs are all in the book table, but the genre is
an ID number that references the genre table. So, how can we get those Mystery IDs,
and use those IDs to filter the book table? There are several ways to do this!*/

-- SUBQUERY METHOD - First get the Mystery IDs, then use those IDs to filter the Book table
-- Total execution time: 00:00:00.172
    SELECT b.title, b.isbn
    FROM staffuser.book AS b 
    WHERE b.genre_id IN     
        (SELECT genre_id    
        FROM staffuser.genre 
        WHERE genres LIKE 'Mystery');


-- INNER JOIN METHOD - Filter the Genre table down to only Mystery IDs, then join the results to
    -- the book table to get only the books where Genre ID is in the mystery category
    -- Total execution time: 00:00:00.146
    SELECT b.title, b.isbn
    FROM staffuser.book AS b
    INNER JOIN staffuser.genre AS g
    ON b.genre_id = g.genre_id
    WHERE g.genres LIKE 'Mystery';

/* Inner join runs in about 85% of the time it takes to run the subquery method, so
if we were writing this query to run on a very large dataset or it were a query we knew
would run very frequently, it'd be wise to choose the inner join method instead of the
subquery method. If it's a query you're running just one time for an analysis and your
dataset isn't massive, this sort of efficiency analysis is less important */

Commands completed successfully.

(8 rows affected)

Total execution time: 00:00:00.146

title,isbn
And Then There Were None,335973357
Still Life,455128657
A Fatal Grace,832517012
The Cruelest Month,419285319
The Murder Stone,656621400
The Brutal Telling,144255852
Bury Your Dead,208180961
A Trick of the Light,93451531


In [12]:
-- Query 2 -- Return all of the titles and the author's first and last names 
            --for books written by authors who are currently living.

/* If we examine our author data, we can see that living authors have
NULL values recorded in the death_year column: */

SELECT first_name, last_name, birth_year, death_year FROM staffuser.author;

/* So, if we want to return the first and last names of the authors who are
currently alive, we can simply filter out any records with data in the
death_year column: */

SELECT first_name, last_name 
FROM staffuser.author
WHERE death_year IS NULL;

/* The request is also asking for book titles, which we can only get from the book table,
Unlike the Mystery genre exercise, where we only needed to return data from one table,
in this instance we want to return data from both the author table AND the book table.
Since we need data from both, we're going to use a join instead of a subquery. */

SELECT b.title, a.first_name, a.last_name -- get the title from the book table and first/last name from the author table
FROM staffuser.book AS b -- alias your tables to make your life easier
INNER JOIN staffuser.author AS a -- inner join so we only keep things that match
ON b.author_id = a.author_id -- tell SQL which columns to match on
WHERE a.death_year IS NULL; -- and apply our filter

(19 rows affected)

Total execution time: 00:00:00.080

title,first_name,last_name
The Golden Compass,Phillip,Pullman
The Subtle Knife,Phillip,Pullman
The Amber Spyglass,Phillip,Pullman
The Alchemist,Paulo,Coelho
The Joy Luck Club,Amy,Tan
The Moon Lady,Amy,Tan
The Handmaids Tale,Margaret,Atwood
The Testaments,Margaret,Atwood
Still Life,Louise,Penny
A Fatal Grace,Louise,Penny


## **Part C: Loan Out a Book**

## 

The library would like you to create a query that will update the database when a book is loaned out.  

This query needs to perform the following functions:

1. Change **available** to **0** (false) for the appropriate book.
    
2. Add a new row to the **loan** table with today's date as the **date\_out** and the ids in the row matching the appropriate **patron\_id** and **book\_id**.
    
3. Update the appropriate **patron** with the **loan\_id** for the new row created in the **loan** table.
    

You can use any patron and book that strikes your fancy to create and test out this query.'

> _Note:  Every time you test this, you will add an additional row to the output, creating a new loan id each time.  If you do not update the patron id, then a patron will have multiple loan ids.  This is not an issue for Part C;_ _however, this may cause issues when you work on Part F._

[https://stackoverflow.com/questions/224732/sql-update-from-one-table-to-another-based-on-a-id-match](https://stackoverflow.com/questions/224732/sql-update-from-one-table-to-another-based-on-a-id-match)

[https://docs.microsoft.com/en-us/sql/t-sql/queries/update-transact-sql?view=sql-server-ver15](https://docs.microsoft.com/en-us/sql/t-sql/queries/update-transact-sql?view=sql-server-ver15)

In [35]:
-- For the sake of example, we're going to use book_id 10, "Sense and Sensibility"
SELECT * from staffuser.book where book_id = 10 -- Note that the available column is currently 1, 
                                                -- meaning it's available to checkout
-- And patron_id 5, "Wren Gergler"
SELECT * from staffuser.patron where patron_id = 5 

-- /* Wren is going to check out Sense and Sensibility, so we need to change that book's availability
-- to 0. */

UPDATE staffuser.book
SET available = 0
WHERE book_id = 10; 

SELECT * from staffuser.book where book_id = 10

/* Now, we're going to document that Wren checked the book out on the loan table. To do this, we will
create a new row on the loan table with Wren's patron ID, Sense and Sensibility's book ID, and we'll
document that the book was checked out today in the date_out column. */

INSERT INTO staffuser.loan (patron_id, date_out, book_id)
VALUES (5,  -- Wren's patron ID
        GETDATE(), -- today's date
        10); -- Sense and Sensibility's book ID 

SELECT * FROM staffuser.loan;


/* Now we need to get the ID we just generated by loaning out Sense and Sensibility to Wren and
add it to the patron table so we can cross-reference. */

UPDATE staffuser.patron
SET staffuser.patron.loan_id = (
    SELECT staffuser.loan.loan_id
    FROM staffuser.loan
    WHERE staffuser.loan.patron_id = staffuser.patron.patron_id
);

SELECT * FROM staffuser.patron WHERE patron_id = 5;

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

book_id,author_id,title,isbn,available,genre_id
10,6,Sense and Sensibility,156879860,1,2


patron_id,first_name,last_name,loan_id
5,Wren,Gergler,NULL


book_id,author_id,title,isbn,available,genre_id
10,6,Sense and Sensibility,156879860,0,2


loan_id,patron_id,date_out,date_in,book_id
1,5,2023-08-29,NULL,10


patron_id,first_name,last_name,loan_id
5,Wren,Gergler,1


Check out five more books to five more patrons

In [36]:
-- Patron 1, Book 1

UPDATE staffuser.book
SET available = 0
WHERE book_id = 1; 

INSERT INTO staffuser.loan (patron_id, date_out, book_id)
VALUES (1,  -- patron ID
        GETDATE(), -- today's date
        1); -- book ID 

UPDATE staffuser.patron
SET staffuser.patron.loan_id = (
    SELECT staffuser.loan.loan_id
    FROM staffuser.loan
    WHERE staffuser.loan.patron_id = staffuser.patron.patron_id
);

-- Patron 2, Book 2

UPDATE staffuser.book
SET available = 0
WHERE book_id = 2; 

INSERT INTO staffuser.loan (patron_id, date_out, book_id)
VALUES (2,  -- patron ID
        GETDATE(), -- today's date
        2); -- book ID 

UPDATE staffuser.patron
SET staffuser.patron.loan_id = (
    SELECT staffuser.loan.loan_id
    FROM staffuser.loan
    WHERE staffuser.loan.patron_id = staffuser.patron.patron_id
);

-- Patron 3, Book 3

UPDATE staffuser.book
SET available = 0
WHERE book_id = 3; 

INSERT INTO staffuser.loan (patron_id, date_out, book_id)
VALUES (3,  -- patron ID
        GETDATE(), -- today's date
        3); -- book ID 

UPDATE staffuser.patron
SET staffuser.patron.loan_id = (
    SELECT staffuser.loan.loan_id
    FROM staffuser.loan
    WHERE staffuser.loan.patron_id = staffuser.patron.patron_id
);

-- Patron 4, Book 4

UPDATE staffuser.book
SET available = 0
WHERE book_id = 4; 

INSERT INTO staffuser.loan (patron_id, date_out, book_id)
VALUES (4,  -- patron ID
        GETDATE(), -- today's date
        4); -- book ID 

UPDATE staffuser.patron
SET staffuser.patron.loan_id = (
    SELECT staffuser.loan.loan_id
    FROM staffuser.loan
    WHERE staffuser.loan.patron_id = staffuser.patron.patron_id
);

-- Patron 5, Book 5

UPDATE staffuser.book
SET available = 0
WHERE book_id = 6; 

INSERT INTO staffuser.loan (patron_id, date_out, book_id)
VALUES (6,  -- patron ID
        GETDATE(), -- today's date
        6); -- book ID 

UPDATE staffuser.patron
SET staffuser.patron.loan_id = (
    SELECT staffuser.loan.loan_id
    FROM staffuser.loan
    WHERE staffuser.loan.patron_id = staffuser.patron.patron_id
);

(1 row affected)

(1 row affected)

(48 rows affected)

(1 row affected)

(1 row affected)

...

(48 rows affected)

(1 row affected)

(1 row affected)

(48 rows affected)

Total execution time: 00:00:00.361

In [44]:
SELECT b.title, 
    a.first_name + ' ' + a.last_name AS 'author', 
    p.first_name + ' ' + p.last_name AS 'patron', 
    l.date_out, l.date_in 
FROM staffuser.loan AS l
JOIN staffuser.book AS b on b.book_id = l.book_id
JOIN staffuser.author AS a on a.author_id = b.author_id
JOIN staffuser.patron AS p on p.patron_id = l.patron_id;

(6 rows affected)

Total execution time: 00:00:00.101

title,author,patron,date_out,date_in
Sense and Sensibility,Jane Austin,Wren Gergler,2023-08-29,NULL
A Midsummers Night Dream,William Shakespeare,Jennifer Slayny,2023-08-29,NULL
Romeo and Juliet,William Shakespeare,Susan Traviss,2023-08-29,NULL
The Golden Compass,Phillip Pullman,Vincent Ritelli,2023-08-29,NULL
The Subtle Knife,Phillip Pullman,Tasia Laybourne,2023-08-29,NULL
The Alchemist,Paulo Coelho,Tory Lees,2023-08-29,NULL


## **Part D: Check a Book Back In**

Work with the same patron and book as you create your query to check a book back into the library.  

The check a book back in, your query need to do the following:

1. Change **available** to **1** (true) for the appropriate book.
    
2. Update the appropriate row in the **<mark><mark><mark>loan</mark></mark></mark>** table with today's date as the **date\_in**.
    
3. Update the appropriate **patron** changing **<mark><mark><mark>loan</mark></mark></mark>\_id** back to **NULL**.

In [27]:
/* Now we're going to check Wren's book back into the library. */

-- -- Mark the book "available" again in the book table- remember, our book ID is 10
-- SELECT * FROM staffuser.book WHERE book_id = 10;

-- UPDATE staffuser.book
-- SET available = 1
-- WHERE book_id = 10;

-- SELECT * FROM staffuser.book WHERE book_id = 10;

-- -- Update date_in column on the loan table to reflect that the book was returned today
-- SELECT * FROM staffuser.loan WHERE book_id = 10;

-- UPDATE staffuser.loan
-- SET date_in = GETDATE()
-- WHERE date_in IS NULL;

-- SELECT * FROM staffuser.loan WHERE book_id = 10;

-- Lastly, we just have to switch Wren's loan_id to NULL since she no longer has a book out.

-- SELECT * FROM staffuser.patron WHERE patron_id = 5;

-- UPDATE staffuser.patron
-- SET loan_id = NULL
-- WHERE patron_id = 5;

-- SELECT * FROM staffuser.patron WHERE patron_id = 5;

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.506

patron_id,first_name,last_name,loan_id
5,Wren,Gergler,1


patron_id,first_name,last_name,loan_id
5,Wren,Gergler,NULL


## **Part E: Create a temp table**

## 

Write a query that returns a temporary table for a patron that shows them all of their loans.

_Question to think about:_  Why might a temp table be a good option for this type of query?

In [65]:
-- TEMP TABLE

/* Start by thinking of what you would want to see in this table if you were a patron 
trying to check on your loans. You'd probably want to see the book's title and when
you checked it out, right? So let's logic through how we'll build this table.

We get our information about books on loan from the loan table.
We get book titles from the book table.
We get information about patrons from the patron table. 
We know a book has not been checked in if the date_in column on the loan table is NULL.

We know that the loan table has a book ID and a patron ID, so we can start with the
loan table and then join our remaining data onto it. 

Lets get all rows from the loan table where:
- Wren is the patron (patron_id = 5)
- The book is currently checked out (date_in = NULL) 

We know we'll be joining this with other tables, so let's go ahead
and start writing aliases now. */

-- SELECT * 
-- FROM staffuser.loan AS l
-- WHERE l.patron_id = 5 AND l.date_in IS NULL;

/* Now let's get Wren's actual name into the table instead of just their ID. To make
it easier for our end user to read, let's combine the first_name and last_name
columns from the patron table into a single column called "patron." */

-- SELECT  p.first_name + ' ' + p.last_name AS 'patron', 
--         l.*
-- FROM staffuser.patron AS p
-- INNER JOIN staffuser.loan AS l ON p.patron_id = l.patron_id
-- WHERE p.patron_id = 5 AND l.date_in IS NULL;

/* Now we'll add the book title from the book table. */

-- SELECT  p.first_name + ' ' + p.last_name AS 'patron', 
--         b.title, 
--         l.*
-- FROM staffuser.patron AS p
-- INNER JOIN staffuser.loan AS l ON p.patron_id = l.patron_id
-- INNER JOIN staffuser.book AS b on l.book_id = b.book_id
-- WHERE p.patron_id = 5 AND l.date_in IS NULL;

/* Let's get rid of the unnecessary columns from the loan table. All Wren needs
to see from that table is the checkout date for each book. We can still perform
joins on the ID columns even if we can't see them on the final table!*/

-- SELECT  p.first_name + ' ' + p.last_name AS 'patron', 
--         b.title, 
--         l.date_out
-- FROM staffuser.patron AS p
-- INNER JOIN staffuser.loan AS l ON p.patron_id = l.patron_id
-- INNER JOIN staffuser.book AS b on l.book_id = b.book_id
-- WHERE p.patron_id = 5 AND l.date_in IS NULL;

/* We may need to query this table later. However, we don't want to create
and save this new table EVERY time a different patron looks at their data, right?
So, we can create Temp Table that can be queried inside of this SQL document without
rewriting the ENTIRE query every time we want to use that table. */

-- DROP TABLE IF EXISTS #TempTest1;

-- SELECT  p.first_name + ' ' + p.last_name AS 'patron', 
--         b.title, 
--          b.author,
--         l.date_out
-- INTO #TempTest1
-- FROM staffuser.patron AS p
-- INNER JOIN staffuser.loan AS l ON p.patron_id = l.patron_id
-- INNER JOIN staffuser.book AS b on l.book_id = b.book_id
-- WHERE p.patron_id = 5 AND l.date_in IS NULL;

/* Look over at the Tables section in your database navigator- "#TempTest1" doesn't exist there.
It only exists in this session. But, you can still query it.*/

SELECT COUNT(*) AS 'Total # of books currently checked out' FROM #TempTest1;

/* Imagine you're building a library app- it might be useful to have a
query like this that any user can access in order to see the books
they currently have checked out- in fact, this is exactly the kind of query
your library likely uses to show you YOUR checked-out books when you check
your account.


(1 row affected)

Total execution time: 00:00:00.550

Total books currently checked out
1


## **Part F: Wrap-up Query**

## 

Create a query that returns the names of the patrons with the genre of every book they currently have checked out.

Things to consider:  How are these tables related and connected to each other?

> _Note:  If you find a NULL in your output instead of a loan id,   this is likely because there are multiple loans per patron.  Try_ _clearing out the loan table.  You can do this by either dropping the entire table or clearing specific rows._

In [68]:

-- retrieve first and last name of patron, and genre of loaned out book

SELECT first_name, last_name, genres
FROM (
    SELECT first_name, last_name, book_id FROM staffuser.patron -- return first and last name, and book id from patron_loan table
    INNER JOIN staffuser.loan AS l ON l.loan_id = patron.loan_id -- Merge entries from loan and patron tables with same loan_ids
    ) AS patron_loan  -- alias of our joined tables
INNER JOIN (
    SELECT book_id, genre_id FROM staffuser.book
    WHERE available = 0 -- Return entries for books that are NOT available.
    ) AS checked_out_genres ON patron_loan.book_id = checked_out_genres.book_id -- Merge entries from patron_loan and checked_out_genres that have mathcing book_id values.
INNER JOIN staffuser.genre ON checked_out_genres.genre_id = genre.genre_id; -- Merge entries from genre and checked_out_genres that have mathcing genre_id values.



/*
Note:  If you find a NULL in your output instead of a loan id,
 this is likely because there are multiple loans per patron.  
 Try clearing out the loan table.  
 You can do this by either dropping the entire table or clearing specific rows.
 */

(6 rows affected)

Total execution time: 00:00:00.248

first_name,last_name,genres
Jennifer,Slayny,Humor
Susan,Traviss,Romance
Vincent,Ritelli,Fantasy
Tasia,Laybourne,Fantasy
Wren,Gergler,Adventure
Tory,Lees,Adventure


## BONUS CHALLENGE

In [87]:
/*

UPDATE JunkCF.courtney.book
SET available = CASE
    WHEN genre_id = 25 THEN 1
    ELSE 0
    END
WHERE JunkCF.courtney.book.book_id = 10;

-- This is one way this may be accomplished. The focus on the final bonus mission is really more on thinking through the problem and investigating online to see if it can be done. If someone codes something, that is great!

*/

Commands completed successfully.

Total execution time: 00:00:00.052